# 导入所需要的库

In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
import math
import warnings
warnings.filterwarnings('ignore')

# 数据处理

## 导入上方代码块处理好后的DATA1

In [2]:
DATA1=pd.read_excel('../data with adjustment/9-12fp.xlsx')
#print(len(DATA1))
#打印OD列的集合
#print(len(DATA1['OD'].unique()))
#打印联程列值为1的个数
#print(len(DATA1[DATA1['联程']==1]))
#打印联程列值不为1和0的个数
#print(len(DATA1[DATA1['联程']!=1][DATA1['联程']!=0]))
#更换列名，把联程换为liancheng
DATA1.rename(columns={'联程':'liancheng'},inplace=True)
#print(DATA1)

## 选取9-12点的航班计划数据

In [3]:
#T是时间片的对应字典;ST是模型启动的时刻，即空域状态改变或有重大保障活动时启动;ET是模型优化的时间区间的终止时刻
ST=108  #本文的时刻优化模型是从9点开始的，即时刻108
ET=144  #本文的时刻优化模型是到12点结束的，即时刻144
#将DATA1按照Time列的值，拆分成大于等于2018/8/1 9:00:00的部分和小于2018/8/1 9:00:00的部分
#DATA1_1是未起飞的航班WF
DATA1_1=DATA1[DATA1['Time']>=ST]
#print(len(DATA1_1))
#print(DATA1_1.head(6))
#DATA1_2是已经起飞的航班FF
DATA1_2=DATA1[DATA1['Time']<ST]
#print(len(DATA1_2))
#print(DATA1_2.head(6))

In [4]:
#建立航班集合F
F_flight=set(DATA1['flightid'])
#F_flight转化为列表
F_flight=list(F_flight)
#print(F_flight)
#print(len(F_flight))
#建立航班的OD机场对集合
OD_airport=set(DATA1['OD'])
#OD_airport转化为列表
OD_airport=list(OD_airport)
#print(OD_airport)
#print(len(OD_airport))

## 导入扇区路径DATA2&扇区最小飞行时间DATA3

In [5]:
#导入航班的扇区路径文件【航班OD机场对对应的扇区集合S、航班经过的扇区集合数据】
DATA2=pd.read_excel('../data with adjustment/shanqulujing.xlsx')
#print(len(set(DATA2['flightid'])))
#将DATA2的进扇时间5min、出扇时间5min的列名改为jinshan5min、chushan5min
DATA2.rename(columns={'进扇时间5min':'jinshan5min','出扇时间5min':'chushan5min','进扇时间':'jinshanshijian','出扇时间':'chushanshijian',
                      '进扇高度':'jinshangaodu','出扇高度':'chushangaodu','进扇速度':'jinshansudu','出扇速度':'chushansudu'},inplace=True)
#得到DATA1的flightid列的集合
DATA1_flightid=set(DATA1['flightid'])
#DATA2中只保留flightid列值在DATA1_flightid中的行
DATA2=DATA2[DATA2['flightid'].isin(DATA1_flightid)]
#print(len(set(DATA2['flightid'])))
#print(len(DATA2))
#print(DATA2.head(6))

#导入扇区的最小飞行时间
DATA3=pd.read_excel('../data with adjustment/shanquflytimemin.xlsx')
DATA3.loc[DATA3['MIN']==0,'MIN']=1
print(DATA3[DATA3['sector']=='AP01'])
#print(len(DATA3))

  sector  MIN
0   AP01    1


## 将中文扇区名用英文替代

In [6]:
#得到DATA2的sector列的集合
DATA2_sector=set(DATA2['sector'])
#得到DATA2_sector中包含中文的元素
DATA2_sector_chinese=[i for i in DATA2_sector if '\u4e00' <= i <= '\u9fff']
print(DATA2_sector_chinese)
print(len(DATA2_sector_chinese))
#建立字典，key为DATA2_sector_chinese中的元素，value为key的汉语拼音
DATA2_sector_chinese_pinyin={'广州管制区':'guangzhouguanzhiqu','桂林进近':'guilinjinjin','呼和浩特管制区':'huhehaoteguanzhiqu','三亚进近':'sanyajinjin','郑州管制区':'zhengzhouguanzhiqu'
                             ,'湛江02扇区':'zhanjiang02shanqu','宁波进近':'linbojinjin','合肥管制区':'hefeiguanzhiqu','南昌管制区':'nanchangguanzhiqu','湛江01扇区':'zhanjiang01shanqu'
                             ,'桂林管制区':'guilinguanzhiqu','武汉管制区':'wuhanguanzhiqu','长沙管制区':'changshaguanzhiqu','湛江03扇区':'zhanjiang03shanqu','北京进近':'beijingjinjin'
                             ,'厦门管制区':'xiamenguanzhiqu','太原管制区':'taiyuanguanzhiqu'}
print(len(DATA2_sector_chinese_pinyin))
#将DATA2中sector列的值中的中文替换为拼音，使用字典DATA2_sector_chinese_pinyin
DATA2['sector']=DATA2['sector'].replace(DATA2_sector_chinese_pinyin)
print(len(DATA2))

#得到DATA3的sector列的集合
DATA3_sector=set(DATA3['sector'])
#得到DATA3_sector中包含中文的元素
DATA3_sector_chinese=[i for i in DATA3_sector if '\u4e00' <= i <= '\u9fff']
print(DATA3_sector_chinese)
print(len(DATA3_sector_chinese))
#建立字典，key为DATA2_sector_chinese中的元素，value为key的汉语拼音
DATA3_sector_chinese_pinyin={'广州管制区':'guangzhouguanzhiqu','桂林进近':'guilinjinjin','呼和浩特管制区':'huhehaoteguanzhiqu','三亚进近':'sanyajinjin','郑州管制区':'zhengzhouguanzhiqu'
                             ,'湛江02扇区':'zhanjiang02shanqu','宁波进近':'linbojinjin','合肥管制区':'hefeiguanzhiqu','南昌管制区':'nanchangguanzhiqu','湛江01扇区':'zhanjiang01shanqu'
                             ,'桂林管制区':'guilinguanzhiqu','武汉管制区':'wuhanguanzhiqu','长沙管制区':'changshaguanzhiqu','湛江03扇区':'zhanjiang03shanqu','北京进近':'beijingjinjin'
                             ,'厦门管制区':'xiamenguanzhiqu','太原管制区':'taiyuanguanzhiqu','杭州进近':'hangzhoujinjin','乌鲁木齐进近':'wulumuqijinjin','汕头进近':'shantoujinjin'
                             ,'武汉进近':'wuhanjinjin','珠海终端区':'zhuhaizhongduanqu','南京进近':'nanjingjinjin','南昌进近':'nanchangjinjin','昆明进近':'kunmingjinjin'}
print(len(DATA3_sector_chinese_pinyin))
#对DATA3进行同样的操作
DATA3['sector']=DATA3['sector'].replace(DATA3_sector_chinese_pinyin)
print(len(DATA3))

['呼和浩特管制区', '郑州管制区', '武汉管制区', '太原管制区', '宁波进近', '桂林管制区', '湛江03扇区', '北京进近', '三亚进近', '厦门管制区', '湛江02扇区', '湛江01扇区', '合肥管制区', '长沙管制区', '广州管制区', '南昌管制区', '桂林进近']
17
17
36742
['昆明进近', '珠海终端区', '桂林管制区', '南京进近', '湛江01扇区', '合肥管制区', '长沙管制区', '南昌管制区', '呼和浩特管制区', '武汉管制区', '南昌进近', '太原管制区', '湛江03扇区', '北京进近', '厦门管制区', '湛江02扇区', '广州管制区', '三亚进近', '乌鲁木齐进近', '桂林进近', '郑州管制区', '武汉进近', '宁波进近', '杭州进近', '汕头进近']
25
25
1298


## 得到WF、FF航班的扇区路径

In [7]:
#构造航班号flightid与飞行路径sector的关系字典flight_sector【已经按照时间顺序排好了飞机进入扇区的顺序】
#对于所有航班，flight_sector_WFFF
flight_sector_WFFF={}
for f in F_flight:
#f为flightid，找到flightid为f对应的所有sector，将sector形成一个列表
    flight_sector_WFFF[f]=DATA2[DATA2['flightid']==f]['sector'].tolist()
    
flight_sector_WF={}
#循环DATA1_1中flightid列的所有航班号
for f in DATA1_1['flightid']:
#f为flightid，找到flightid为f对应的所有sector，将sector形成一个列表
    flight_sector_WF[f]=DATA2[DATA2['flightid']==f]['sector'].tolist()
    
flight_sector_FF={}
#循环DATA1_2中flightid列的所有航班号
for f in DATA1_2['flightid']:
#f为flightid，找到flightid为f对应的所有sector，将sector形成一个列表
    flight_sector_FF[f]=DATA2[DATA2['flightid']==f]['sector'].tolist()

In [8]:
ARR_jichang=pd.read_csv('../data with adjustment/ARR1.csv')
ARR_airport=ARR_jichang['jichang'].unique()
ARR_airport

array(['ZBAA', 'ZBHH', 'ZBSJ', 'ZBTJ', 'ZBYN', 'ZGGG', 'ZGHA', 'ZGNN',
       'ZGSD', 'ZGSZ', 'ZHCC', 'ZHHH', 'ZJHK', 'ZJSY', 'ZLLL', 'ZLXY',
       'ZPPP', 'ZSAM', 'ZSCN', 'ZSFZ', 'ZSHC', 'ZSJN', 'ZSNB', 'ZSNJ',
       'ZSOF', 'ZSPD', 'ZSQD', 'ZSSS', 'ZSWZ', 'ZSYT', 'ZUCK', 'ZUGY',
       'ZUUU', 'ZWWW', 'ZYCC', 'ZYHB', 'ZYTL', 'ZYTX'], dtype=object)

## ARR的不确定性容量

In [9]:
#下面计算【降落机场的不确定与确定容量】【这类计算不受参与模型运算的航班数据影响，独立】
#定义用来存储最终结果的dataframe
capacity_arr_zuizhongjieguo=pd.DataFrame()
CAPICITY_ARR_zuizhongjieguo=pd.DataFrame()
#得到需要考虑容量限制的机场
ARR_jichang=pd.read_csv('../data with adjustment/ARR1.csv')
#print(len(ARR_jichang))

#只保留时间片15数值为36-60的行
ARR_jichang=ARR_jichang[ARR_jichang['shijian'].isin([36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60])]
ARR_airport=ARR_jichang['jichang'].unique()
#print(len(ARR_airport))
for ii in ARR_airport:
    for j in range(36,61):
        #得到机场i在时刻j的流量数据列表
        ARR_jichang_IJ=ARR_jichang[(ARR_jichang['jichang']==ii)&(ARR_jichang['shijian']==j)].values.tolist()
        #用ARR_jichang_IJ_LIST来存储ARR_jichang_IJ中每个列表元素的第3个值【不同天同一机场、同一时间段的各种流量数】
        ARR_jichang_IJ_LIST=[]
        for k in ARR_jichang_IJ:
            ARR_jichang_IJ_LIST.append(k[3])
        #print(len(ARR_jichang_IJ_LIST))
        #print(ARR_jichang_IJ_LIST)
        #依据1-50天的历史流量计算出进场容量的确定性值【后续会进行公布容量的替换】
        #把ARR_jichang_IJ_LIST中的NAN值去掉
        #ARR_jichang_IJ_LIST=[i for i in ARR_jichang_IJ_LIST if str(i)!='NAN']
        #85%分位数作为确定性容量
        airport_arr_capacity_36_certain=np.percentile(ARR_jichang_IJ_LIST,85)
        #airport_arr_capacity_36_certain=np.mean(ARR_jichang_IJ_LIST)#这个不合适，公布容量一般是历史流量的85%分位数
        #print(airport_arr_capacity_36_certain)
        airport_arr_capacity_36_value=set(ARR_jichang_IJ_LIST)
        airport_arr_capacity_36_value_count=[]
        for i in airport_arr_capacity_36_value:
            airport_arr_capacity_36_value_count.append(ARR_jichang_IJ_LIST.count(i))
            #转化为概率
            airport_arr_capacity_36_value_count_P=[i/sum(airport_arr_capacity_36_value_count) for i in airport_arr_capacity_36_value_count]
            P=airport_arr_capacity_36_value_count_P.copy()
            #转化成累计概率，airport_arr_capacity_36_value_count_P的每一个值都加上其后面所有的值
            for h in range(len(airport_arr_capacity_36_value_count_P)-1):
                airport_arr_capacity_36_value_count_P[h]=sum(airport_arr_capacity_36_value_count_P[h:])
        B=airport_arr_capacity_36_value.copy()
        B=list(B)
        C=len(B)
        #建立长度为C的列表，值为ii
        D=[ii for i in range(C)]
        #建立长度为C的列表，值为j
        E=[j for i in range(C)]
        #建立长度为C的列表，值为airport_arr_capacity_36_certain
        F=[airport_arr_capacity_36_certain for i in range(C)]
        #建立长度为C的列表，值为airport_arr_capacity_36_value_count_P
        G=[1 for i in range(C)]
        #现在有机场、时刻、确定性容量、确定性容量概率、不确定性容量、不确定性容量概率、累加概率，可以将它们填入CAPICITY_ARR_zuizhongjieguo中
        CAPICITY_ARR_zuizhongjieguo['jichang']=D
        CAPICITY_ARR_zuizhongjieguo['shike']=E
        CAPICITY_ARR_zuizhongjieguo['quedingxingrongliang']=F
        CAPICITY_ARR_zuizhongjieguo['quedingxingronglianggailv']=G
        CAPICITY_ARR_zuizhongjieguo['buquedingxingrongliang']=B
        CAPICITY_ARR_zuizhongjieguo['buquedingxingronglianggailv']=P
        CAPICITY_ARR_zuizhongjieguo['leijiagailv']=airport_arr_capacity_36_value_count_P
        #合并dataframe并刷新行索引
        #将A与CAPICITY_ARR进行合并
        capacity_arr_zuizhongjieguo=pd.concat([capacity_arr_zuizhongjieguo,CAPICITY_ARR_zuizhongjieguo],axis=0)
        #刷新Dataframe的索引
        capacity_arr_zuizhongjieguo.index=[i for i in range(len(capacity_arr_zuizhongjieguo))]
        #将CAPICITY_ARR清空
        CAPICITY_ARR_zuizhongjieguo=pd.DataFrame()
#capacity_arr_zuizhongjieguo的确定性容量列是否有空值
#print(capacity_arr_zuizhongjieguo['确定性容量'].isnull().any())
#将capacity_arr_zuizhongjieguo确定性容量列的值进行向上取整
capacity_arr_zuizhongjieguo['quedingxingrongliang']=capacity_arr_zuizhongjieguo['quedingxingrongliang'].apply(lambda x:math.ceil(x))
#设置不考虑节点重要度的机会约束值
capacity_arr_zuizhongjieguo[1]=0.95
capacity_arr_zuizhongjieguo[2]=0.92
capacity_arr_zuizhongjieguo[3]=0.89
capacity_arr_zuizhongjieguo[4]=0.86
capacity_arr_zuizhongjieguo[5]=0.83

#计算考虑节点重要度的机会约束值
airport_HBL_importance=pd.read_excel('../data with adjustment/liuliang.xlsx')
#把第一列删除、最后一列删除
airport_HBL_importance=airport_HBL_importance.drop(airport_HBL_importance.columns[0],axis=1)
airport_HBL_importance=airport_HBL_importance.drop(airport_HBL_importance.columns[-1],axis=1)
#只保留ARR_jichang中的机场
airport_HBL_importance=airport_HBL_importance[airport_HBL_importance['jichang'].isin(ARR_airport)]
#将airrport_HBL_importance按照rijunhangbanliang由大到小排序
airport_HBL_importance=airport_HBL_importance.sort_values(by='rijunliuliang',ascending=False)
#重置索引
airport_HBL_importance=airport_HBL_importance.reset_index(drop=True)
#print(len(airport_HBL_importance))
#print(airport_HBL_importance)
#添加新的5列，用来存储机会约束值，名为6-10
airport_HBL_importance[6]=0
airport_HBL_importance[7]=0
airport_HBL_importance[8]=0
airport_HBL_importance[9]=0
airport_HBL_importance[10]=0
#将列6的第一个值赋值为0.7，列7的第一个值赋值为0.65，列8的第一个值赋值为0.6，列9的第一个值赋值为0.55，列10的第一个值赋值为0.5
airport_HBL_importance[6][0]=0.95
airport_HBL_importance[7][0]=0.92
airport_HBL_importance[8][0]=0.89
airport_HBL_importance[9][0]=0.86
airport_HBL_importance[10][0]=0.83
#对于列6与HBL列，用首行(0.75,1636)与末行(0.00,201)进行线性插值，得到列6其他行的机会约束值
for i in range(6,11):
    for j in range(1,38):
        #尾部加上0.000001是为了防止出现0的情况，0会导致后面的计算出现问题，会影响容量转化为机会约束值的计算
        airport_HBL_importance[i][j]=0.8+(airport_HBL_importance[i][0]-0.8)*(airport_HBL_importance['rijunliuliang'][j]-airport_HBL_importance['rijunliuliang'][37])/(
            airport_HBL_importance['rijunliuliang'][0]-airport_HBL_importance['rijunliuliang'][37])+0.000001
#print(airport_HBL_importance[airport_HBL_importance['jichang']=='ZYTX'])
#先复制一份留给CAPICITY_ARR_zuizhongjieguo使用
airport_HBL_importance_copy=airport_HBL_importance.copy()
#导入capactiy_arr_zuizhongjieguo
for i in range(6,11):
    #建立airport与6的一一对应关系
    airport_HBL_importance_dict=dict(zip(airport_HBL_importance['jichang'],airport_HBL_importance[i]))
    #print(airport_HBL_importance_dict)
    #给CAPICITY_DEP_zuizhongjieguo添加名为i的列，按照airport_HBL_importance_dict的机场与机会约束值的对应关系来设置机会约束值
    capacity_arr_zuizhongjieguo[i]=capacity_arr_zuizhongjieguo['jichang'].map(airport_HBL_importance_dict)

#print(P)
#print(airport_arr_capacity_36_value_count_P)
#print(airport_arr_capacity_36_value)    
#print(airport_arr_capacity_36_value_count)
#print(capacity_arr_zuizhongjieguo.head(20))
#判断capacity_arr_zuizhongjieguo中是否存在空值
#print(capacity_arr_zuizhongjieguo.isnull().any())
#输出capacity_arr_zuizhongjieguo机场列的集合
#print(capacity_arr_zuizhongjieguo['机场'].unique())
#print(len(capacity_arr_zuizhongjieguo['机场'].unique()))
capacity_arr_zuizhongjieguo.to_csv('../data with adjustment/A.csv')

## DEP的不确定性容量

In [10]:
#下面计算【降落机场的不确定与确定容量】
#定义用来存储最终结果的dataframe
capacity_dep_zuizhongjieguo=pd.DataFrame()
CAPICITY_DEP_zuizhongjieguo=pd.DataFrame()
#得到需要考虑容量限制的机场
DEP_jichang=pd.read_csv('../data with adjustment/DEP1.csv')
#print(len(DEP_jichang))

#只保留时间片15数值为36-60的行
DEP_jichang=DEP_jichang[DEP_jichang['shijian'].isin([36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60])]
DEP_airport=DEP_jichang['jichang'].unique()
#print(len(DEP_airport))
for ii in DEP_airport:
    for j in range(36,61):
        #得到机场i在时刻j的流量数据列表
        DEP_jichang_IJ=DEP_jichang[(DEP_jichang['jichang']==ii)&(DEP_jichang['shijian']==j)].values.tolist()
        #用DEP_jichang_IJ_LIST来存储DEP_jichang_IJ中每个列表元素的第3个值【不同天同一机场、同一时间段的各种流量数】
        DEP_jichang_IJ_LIST=[]
        for k in DEP_jichang_IJ:
            DEP_jichang_IJ_LIST.append(k[3])
        #print(len(DEP_jichang_IJ_LIST))
        #print(DEP_jichang_IJ_LIST)
        #依据1-50天的历史流量计算出进场容量的确定性值【后续会进行公布容量的替换】
        #把DEP_jichang_IJ_LIST中的NAN值去掉
        #DEP_jichang_IJ_LIST=[i for i in DEP_jichang_IJ_LIST if str(i)!='NAN']
        #85%分位数作为确定性容量
        airport_dep_capacity_36_certain=np.percentile(DEP_jichang_IJ_LIST,85)
        #airport_dep_capacity_36_certain=np.mean(DEP_jichang_IJ_LIST)#这个不合适，公布容量一般是历史流量的85%分位数
        #print(airport_dep_capacity_36_certain)
        airport_dep_capacity_36_value=set(DEP_jichang_IJ_LIST)
        airport_dep_capacity_36_value_count=[]
        for i in airport_dep_capacity_36_value:
            airport_dep_capacity_36_value_count.append(DEP_jichang_IJ_LIST.count(i))
            #转化为概率
            airport_dep_capacity_36_value_count_P=[i/sum(airport_dep_capacity_36_value_count) for i in airport_dep_capacity_36_value_count]
            P=airport_dep_capacity_36_value_count_P.copy()
            #转化成累计概率，airport_arr_capacity_36_value_count_P的每一个值都加上其后面所有的值
            for h in range(len(airport_dep_capacity_36_value_count_P)-1):
                airport_dep_capacity_36_value_count_P[h]=sum(airport_dep_capacity_36_value_count_P[h:])
        B=airport_dep_capacity_36_value.copy()
        B=list(B)
        C=len(B)
        #建立长度为C的列表，值为ii
        D=[ii for i in range(C)]
        #建立长度为C的列表，值为j
        E=[j for i in range(C)]
        #建立长度为C的列表，值为airport_arr_capacity_36_certain
        F=[airport_dep_capacity_36_certain for i in range(C)]
        #建立长度为C的列表，值为airport_arr_capacity_36_value_count_P
        G=[1 for i in range(C)]
        #现在有机场、时刻、确定性容量、确定性容量概率、不确定性容量、不确定性容量概率、累加概率，可以将它们填入CAPICITY_DEP_zuizhongjieguo中
        CAPICITY_DEP_zuizhongjieguo['jichang']=D
        CAPICITY_DEP_zuizhongjieguo['shike']=E
        CAPICITY_DEP_zuizhongjieguo['quedingxingrongliang']=F
        CAPICITY_DEP_zuizhongjieguo['quedingxingronglianggailv']=G
        CAPICITY_DEP_zuizhongjieguo['buquedingxingrongliang']=B
        CAPICITY_DEP_zuizhongjieguo['buquedingxingronglianggailv']=P
        CAPICITY_DEP_zuizhongjieguo['leijiagailv']=airport_dep_capacity_36_value_count_P
        #合并dataframe并刷新行索引
        #将A与CAPICITY_ARR进行合并
        capacity_dep_zuizhongjieguo=pd.concat([capacity_dep_zuizhongjieguo,CAPICITY_DEP_zuizhongjieguo],axis=0)
        #刷新Dataframe的索引
        capacity_dep_zuizhongjieguo.index=[i for i in range(len(capacity_dep_zuizhongjieguo))]
        #将CAPICITY_ARR清空
        CAPICITY_DEP_zuizhongjieguo=pd.DataFrame()
#capacity_dep_zuizhongjieguo的确定性容量列是否有空值
#print(capacity_dep_zuizhongjieguo['确定性容量'].isnull().any())
#将capacity_dep_zuizhongjieguo确定性容量列的值进行向上取整
capacity_dep_zuizhongjieguo['quedingxingrongliang']=capacity_dep_zuizhongjieguo['quedingxingrongliang'].apply(lambda x:math.ceil(x))
#设置不考虑节点重要度的机会约束值
capacity_dep_zuizhongjieguo[1]=0.95
capacity_dep_zuizhongjieguo[2]=0.92
capacity_dep_zuizhongjieguo[3]=0.89
capacity_dep_zuizhongjieguo[4]=0.86
capacity_dep_zuizhongjieguo[5]=0.83

#计算考虑节点重要度的机会约束值
airport_HBL_importance=pd.read_excel('../data with adjustment/liuliang.xlsx')
#把第一列删除、最后一列删除
airport_HBL_importance=airport_HBL_importance.drop(airport_HBL_importance.columns[0],axis=1)
airport_HBL_importance=airport_HBL_importance.drop(airport_HBL_importance.columns[-1],axis=1)
#只保留DEP_jichang中的机场
airport_HBL_importance=airport_HBL_importance[airport_HBL_importance['jichang'].isin(DEP_airport)]
#将airrport_HBL_importance按照rijunhangbanliang由大到小排序
airport_HBL_importance=airport_HBL_importance.sort_values(by='rijunliuliang',ascending=False)
#重置索引
airport_HBL_importance=airport_HBL_importance.reset_index(drop=True)
#print(len(airport_HBL_importance))
#print(airport_HBL_importance)
#添加新的5列，用来存储机会约束值，名为6-10
airport_HBL_importance[6]=0
airport_HBL_importance[7]=0
airport_HBL_importance[8]=0
airport_HBL_importance[9]=0
airport_HBL_importance[10]=0
#将列6的第一个值赋值为0.7，列7的第一个值赋值为0.65，列8的第一个值赋值为0.6，列9的第一个值赋值为0.55，列10的第一个值赋值为0.5
airport_HBL_importance[6][0]=0.95
airport_HBL_importance[7][0]=0.92
airport_HBL_importance[8][0]=0.89
airport_HBL_importance[9][0]=0.86
airport_HBL_importance[10][0]=0.83
#对于列6与HBL列，用首行(0.75,1636)与末行(0.00,201)进行线性插值，得到列6其他行的机会约束值
for i in range(6,11):
    for j in range(1,38):
        #尾部加上0.000001是为了防止出现0的情况，0会导致后面的计算出现问题，会影响容量转化为机会约束值的计算
        airport_HBL_importance[i][j]=0.8+(airport_HBL_importance[i][0]-0.8)*(airport_HBL_importance['rijunliuliang'][j]-airport_HBL_importance['rijunliuliang'][37])/(airport_HBL_importance['rijunliuliang'][0]-airport_HBL_importance['rijunliuliang'][37])+0.000001
#print(airport_HBL_importance[airport_HBL_importance['jichang']=='ZYTX'])
#先复制一份留给CAPICITY_DEP_zuizhongjieguo使用
airport_HBL_importance_copy=airport_HBL_importance.copy()
#导入capactiy_arr_zuizhongjieguo
for i in range(6,11):
    #建立airport与6的一一对应关系
    airport_HBL_importance_dict=dict(zip(airport_HBL_importance['jichang'],airport_HBL_importance[i]))
    #print(airport_HBL_importance_dict)
    #给CAPICITY_DEP_zuizhongjieguo添加名为i的列，按照airport_HBL_importance_dict的机场与机会约束值的对应关系来设置机会约束值
    capacity_dep_zuizhongjieguo[i]=capacity_dep_zuizhongjieguo['jichang'].map(airport_HBL_importance_dict)

#print(P)
#print(airport_dep_capacity_36_value_count_P)
#print(airport_dep_capacity_36_value)    
#print(airport_dep_capacity_36_value_count)
#print(capacity_dep_zuizhongjieguo.head(20))
#判断capacity_dep_zuizhongjieguo中是否存在空值
#print(capacity_dep_zuizhongjieguo.isnull().any())
#print(capacity_dep_zuizhongjieguo['机场'].unique())
#print(len(capacity_dep_zuizhongjieguo['机场'].unique()))
capacity_dep_zuizhongjieguo.to_csv('../data with adjustment/D.csv')

## SHANQU的不确定性容量

In [11]:
#下面计算【降落机场的不确定与确定容量】
#定义用来存储最终结果的dataframe
capacity_shanqu_zuizhongjieguo=pd.DataFrame()
CAPICITY_SHANQU_zuizhongjieguo=pd.DataFrame()
#得到需要考虑容量限制的机场
shanqu=pd.read_csv('../data with adjustment/shanqu1.csv')
#print(shanqu)

#只保留时间片15数值为36-60的行
shanqu=shanqu[shanqu['shijian'].isin([36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60])]
shanqu_list=shanqu['shanqu'].unique()
#print(len(shanqu_list))
for ii in shanqu_list:
    for j in range(36,61):
        #得到机场i在时刻j的流量数据列表
        shanqu_IJ=shanqu[(shanqu['shanqu']==ii)&(shanqu['shijian']==j)].values.tolist()
        #用shanqu_IJ_LIST来存储shanqu_IJ中每个列表元素的第3个值【不同天同一机场、同一时间段的各种流量数】
        shanqu_IJ_LIST=[]
        for k in shanqu_IJ:
            shanqu_IJ_LIST.append(k[3])
        #print(len(shanqu_IJ_LIST))
        #print(shanqu_IJ_LIST)
        #依据1-50天的历史流量计算出进场容量的确定性值【后续会进行公布容量的替换】
        #把shanqu_IJ_LIST中的NAN值去掉
        #shanqu_IJ_LIST=[i for i in shanqu_IJ_LIST if str(i)!='NAN']
        #85%分位数作为确定性容量
        shanqu_capacity_36_certain=np.percentile(shanqu_IJ_LIST,85)
        #shanqu_capacity_36_certain=np.mean(shanqu_IJ_LIST)#这个不合适，公布容量一般是历史流量的85%分位数
        #print(shanqu_capacity_36_certain)
        shanqu_capacity_36_value=set(shanqu_IJ_LIST)
        shanqu_capacity_36_value_count=[]
        for i in shanqu_capacity_36_value:
            shanqu_capacity_36_value_count.append(shanqu_IJ_LIST.count(i))
            #转化为概率
            shanqu_capacity_36_value_count_P=[i/sum(shanqu_capacity_36_value_count) for i in shanqu_capacity_36_value_count]
            P=shanqu_capacity_36_value_count_P.copy()
            #转化成累计概率，airport_arr_capacity_36_value_count_P的每一个值都加上其后面所有的值
            for h in range(len(shanqu_capacity_36_value_count_P)-1):
                shanqu_capacity_36_value_count_P[h]=sum(shanqu_capacity_36_value_count_P[h:])
        B=shanqu_capacity_36_value.copy()
        B=list(B)
        C=len(B)
        #建立长度为C的列表，值为ii
        D=[ii for i in range(C)]
        #建立长度为C的列表，值为j
        E=[j for i in range(C)]
        #建立长度为C的列表，值为airport_arr_capacity_36_certain
        F=[shanqu_capacity_36_certain for i in range(C)]
        #建立长度为C的列表，值为airport_arr_capacity_36_value_count_P
        G=[1 for i in range(C)]
        #现在有机场、时刻、确定性容量、确定性容量概率、不确定性容量、不确定性容量概率、累加概率，可以将它们填入CAPICITY_SHANQU_zuizhongjieguo中
        CAPICITY_SHANQU_zuizhongjieguo['shanqu']=D
        CAPICITY_SHANQU_zuizhongjieguo['shike']=E
        CAPICITY_SHANQU_zuizhongjieguo['quedingxingrongliang']=F
        CAPICITY_SHANQU_zuizhongjieguo['quedingxingronglianggailv']=G
        CAPICITY_SHANQU_zuizhongjieguo['buquedingxingrongliang']=B
        CAPICITY_SHANQU_zuizhongjieguo['buquedingxingronglianggailv']=P
        CAPICITY_SHANQU_zuizhongjieguo['leijiagailv']=shanqu_capacity_36_value_count_P
        #合并dataframe并刷新行索引
        #将A与CAPICITY_ARR进行合并
        capacity_shanqu_zuizhongjieguo=pd.concat([capacity_shanqu_zuizhongjieguo,CAPICITY_SHANQU_zuizhongjieguo],axis=0)
        #刷新Dataframe的索引
        capacity_shanqu_zuizhongjieguo.index=[i for i in range(len(capacity_shanqu_zuizhongjieguo))]
        #将CAPICITY_ARR清空
        CAPICITY_SHANQU_zuizhongjieguo=pd.DataFrame()
#capacity_shanqu_zuizhongjieguo的确定性容量列是否有空值
#print(capacity_shanqu_zuizhongjieguo['确定性容量'].isnull().any())
#将capacity_shanqu_zuizhongjieguo确定性容量列的值进行向上取整
capacity_shanqu_zuizhongjieguo['quedingxingrongliang']=capacity_shanqu_zuizhongjieguo['quedingxingrongliang'].apply(lambda x:math.ceil(x))
#设置不考虑节点重要度的机会约束值
capacity_shanqu_zuizhongjieguo[1]=0.95
capacity_shanqu_zuizhongjieguo[2]=0.92
capacity_shanqu_zuizhongjieguo[3]=0.89
capacity_shanqu_zuizhongjieguo[4]=0.86
capacity_shanqu_zuizhongjieguo[5]=0.83

#计算考虑节点重要度的机会约束值
shanqu_jinmidu=pd.read_excel('../data with adjustment/SHANQU1.xlsx')
#把第一列删除、最后一列删除
shanqu_jinmidu=shanqu_jinmidu.drop(shanqu_jinmidu.columns[2],axis=1)
shanqu_jinmidu=shanqu_jinmidu.drop(shanqu_jinmidu.columns[2],axis=1)
shanqu_jinmidu=shanqu_jinmidu.drop(shanqu_jinmidu.columns[2],axis=1)
#只保留shanqu中的机场
shanqu_jinmidu=shanqu_jinmidu[shanqu_jinmidu['shanqu'].isin(shanqu_list)]
#将shanqu_jinmidu按照2du由大到小排序
shanqu_jinmidu=shanqu_jinmidu.sort_values(by='2du',ascending=False)
#重置索引
shanqu_jinmidu=shanqu_jinmidu.reset_index(drop=True)
#print(len(shanqu_jinmidu))
#print(shanqu_jinmidu)
#添加新的5列，用来存储机会约束值，名为6-10
shanqu_jinmidu[6]=0
shanqu_jinmidu[7]=0
shanqu_jinmidu[8]=0
shanqu_jinmidu[9]=0
shanqu_jinmidu[10]=0
#将列6的第一个值赋值为0.7，列7的第一个值赋值为0.65，列8的第一个值赋值为0.6，列9的第一个值赋值为0.55，列10的第一个值赋值为0.5
shanqu_jinmidu[6][0]=0.95
shanqu_jinmidu[7][0]=0.92
shanqu_jinmidu[8][0]=0.89
shanqu_jinmidu[9][0]=0.86
shanqu_jinmidu[10][0]=0.83
#对于列6与HBL列，用首行(0.75,1636)与末行(0.00,201)进行线性插值，得到列6其他行的机会约束值
for i in range(6,11):
    for j in range(1,55):
        #尾部加上0.000001是为了防止出现0的情况，0会导致后面的计算出现问题，会影响容量转化为机会约束值的计算
        shanqu_jinmidu[i][j]=0.8+(shanqu_jinmidu[i][0]-0.8)*(shanqu_jinmidu['2du'][j]-shanqu_jinmidu['2du'][54])/(shanqu_jinmidu['2du'][0]-shanqu_jinmidu['2du'][54])+0.000001
#print(shanqu_jinmidu[shanqu_jinmidu['shanqu']=='ZYTX'])
#先复制一份留给CAPICITY_SHANQU_zuizhongjieguo使用
shanqu_jinmidu_copy=shanqu_jinmidu.copy()
#导入capactiy_arr_zuizhongjieguo
for i in range(6,11):
    #建立airport与6的一一对应关系
    shanqu_jinmidu_dict=dict(zip(shanqu_jinmidu['shanqu'],shanqu_jinmidu[i]))
    #print(shanqu_jinmidu_dict)
    #给CAPICITY_DEP_zuizhongjieguo添加名为i的列，按照airport_HBL_importance_dict的机场与机会约束值的对应关系来设置机会约束值
    capacity_shanqu_zuizhongjieguo[i]=capacity_shanqu_zuizhongjieguo['shanqu'].map(shanqu_jinmidu_dict)

#print(P)
#print(shanqu_capacity_36_value_count_P)
#print(shanqu_capacity_36_value)    
#print(shanqu_capacity_36_value_count)
#print(capacity_shanqu_zuizhongjieguo.tail(20))
#判断capacity_shanqu_zuizhongjieguo中是否存在空值
#print(capacity_shanqu_zuizhongjieguo.isnull().any())
#print(capacity_shanqu_zuizhongjieguo['扇区'].unique())
#print(len(capacity_shanqu_zuizhongjieguo['扇区'].unique()))
capacity_shanqu_zuizhongjieguo.to_csv('../data with adjustment/S.csv')

In [12]:
rongliang1=pd.read_excel('../data with adjustment/shanqurongliang1.xlsx')
rongliang2=pd.read_excel('../data with adjustment/shanqurongliang2.xlsx')
#print(rongliang1)
#print(rongliang2)
#建立ARR的容量字典，key为JI,值为ARR
ARR_rongliang_dict=dict(zip(rongliang1['JI'],rongliang1['ARR']))
#建立DEP的容量字典，key为JI,值为DEP
DEP_rongliang_dict=dict(zip(rongliang1['JI'],rongliang1['DEP']))
#建立SHANQU的容量字典，key为SHANQU,值为CAPACITY
SHANQU_DICT=dict(zip(rongliang2['SHANQU'],rongliang2['CAPACITY']))
#print(len(ARR_rongliang_dict))
#print(ARR_rongliang_dict)
#print(len(DEP_rongliang_dict))
#print(DEP_rongliang_dict)
#print(len(SHANQU_DICT))
#print(SHANQU_DICT)
########################################################################################################################
#用ARR_rongliang_dict的key与capacity_arr_zuizhongjieguo的机场进行比对，用value替换确定性容量
#【被替换的机场为38个中的37个，被替换的扇区为55个中的51个】
for i in ARR_rongliang_dict.keys():
    capacity_arr_zuizhongjieguo.loc[capacity_arr_zuizhongjieguo['jichang']==i,'quedingxingrongliang']=int(ARR_rongliang_dict[i]/4)
#print(capacity_arr_zuizhongjieguo[capacity_arr_zuizhongjieguo['jichang']=='ZBHH'])
#用DEP_rongliang_dict的key与capacity_dep_zuizhongjieguo的机场进行比对，用value替换确定性容量
for i in DEP_rongliang_dict.keys():
    capacity_dep_zuizhongjieguo.loc[capacity_dep_zuizhongjieguo['jichang']==i,'quedingxingrongliang']=int(DEP_rongliang_dict[i]/4)
#print(capacity_dep_zuizhongjieguo[capacity_dep_zuizhongjieguo['jichang']=='ZBHH'])
#用SHANQU_DICT的key与capacity_shanqu_zuizhongjieguo的机场进行比对，用value替换确定性容量
for i in SHANQU_DICT.keys():
    capacity_shanqu_zuizhongjieguo.loc[capacity_shanqu_zuizhongjieguo['shanqu']==i,'quedingxingrongliang']=int(SHANQU_DICT[i]/4)
#print(capacity_shanqu_zuizhongjieguo[capacity_shanqu_zuizhongjieguo['shanqu']=='ZBAAAR04'])

# capacity_shanqu_zuizhongjieguo.to_csv('../新数据提取/A.csv')
# capacity_shanqu_zuizhongjieguo.to_csv('../新数据提取/D.csv')
# capacity_shanqu_zuizhongjieguo.to_csv('../新数据提取/S.csv')
#打印出capacity_arr_zuizhongjieguo中的机场列的集合
#print(capacity_shanqu_zuizhongjieguo['shanqu'].unique())
#print(len(capacity_shanqu_zuizhongjieguo['shanqu'].unique()))

In [13]:
capacity_arr_zuizhongjieguo['jichang']=capacity_arr_zuizhongjieguo['jichang']+'-D'
capacity_dep_zuizhongjieguo['jichang']=capacity_dep_zuizhongjieguo['jichang']+'-O'
#capacity_arr_zuizhongjieguo、capacity_dep_zuizhongjieguo的jichang列只保留前6个字符
capacity_arr_zuizhongjieguo['jichang']=capacity_arr_zuizhongjieguo['jichang'].apply(lambda x:x[:6])
capacity_dep_zuizhongjieguo['jichang']=capacity_dep_zuizhongjieguo['jichang'].apply(lambda x:x[:6])
#print(capacity_arr_zuizhongjieguo['jichang'].unique())
#print(capacity_dep_zuizhongjieguo['jichang'].unique())
# capacity_arr_zuizhongjieguo.to_csv(r'C:\Users\46527\Desktop\A.csv')
# capacity_dep_zuizhongjieguo.to_csv(r'C:\Users\46527\Desktop\D.csv')
# capacity_shanqu_zuizhongjieguo.to_csv(r'C:\Users\46527\Desktop\S.csv')

## 航班：起飞机场

In [14]:
#约束8【航班与起飞机场】
flightid_s_depap={}
for f in DATA1['flightid']:
    flightid_s_depap[f]=DATA1[DATA1['flightid']==f]['s_depap'].values[0]
#print(flightid_s_depap)
#print(len(flightid_s_depap))

## 航班：到达机场

In [15]:
#约束9【航班与到达机场】
flightid_s_arrap={}
for f in DATA1['flightid']:
    flightid_s_arrap[f]=DATA1[DATA1['flightid']==f]['s_arrap'].values[0]
#print(flightid_s_arrap)
#print(len(flightid_s_arrap))

## 航班：扇区列表

In [16]:
#约束2、约束10【所有航班与扇区列表的对应】
flightid_sector={}
for f in DATA1['flightid']:
    flightid_sector[f]=DATA2[DATA2['flightid']==f]['sector'].values.tolist()
print(flightid_sector['B00001'])
#print(len(flightid_sector))

['ZHCC-O', 'ZHCCAR04', 'ZHCCAR02', 'ZHHHAR01', 'ZHHHAR02', 'ZGNNAR01', 'zhanjiang02shanqu', 'ZJHKAP01', 'ZJHKAP02', 'ZJHK-D']


## 需要考虑容量的扇区：其之后的下一个扇区

In [17]:
#约束10【约束10中所考虑扇区的下个扇区】
sector_next={}
for f in DATA1['flightid']:
    for s in shanqu_list:  #这里的shanqu_list是并非所有扇区的列表，而是需要考虑扇区容量的扇区列表
        if s in flightid_sector[f]:#用到了上一步求得的flightid_sector
            sector_next[f,s]=DATA2[DATA2['flightid']==f]['sector'].values.tolist()[DATA2[DATA2['flightid']==f]['sector'].values.tolist().index(s)+1]
        else:
            continue
#print(sector_next)
#print(len(sector_next))

## 航班：扇区列表【不包含最后一个】

In [18]:
#约束5【航班与扇区列表的对应（不附加最后1个）】
flightid_sector_nolastone={}
for f in DATA1['flightid']:
    flightid_sector_nolastone[f]=DATA2[DATA2['flightid']==f]['sector'].values.tolist()[:-1]
#print(flightid_sector_nolastone)
#print(len(flightid_sector_nolastone))

## 航班，航班扇区列表【不包括最后一个】：其后的下一个扇区

In [19]:
#约束5【约束5中所考虑扇区的下个扇区】
sector_NEXT={}
for f in DATA1['flightid']:
    for s in flightid_sector_nolastone[f]:
        sector_NEXT[f,s]=DATA2[DATA2['flightid']==f]['sector'].values.tolist()[DATA2[DATA2['flightid']==f]['sector'].values.tolist().index(s)+1]
#print(sector_NEXT)
#print(len(sector_NEXT))

## 扇区【不包括最后一个】：最小飞行时间

In [20]:
#DATA3只影响这一个地方
#约束5【扇区对应的最小至少停留时间】
sector_TIME={}
for f in DATA1['flightid']:
    for s in flightid_sector_nolastone[f]:
        #s即为data3中的sector，其对应的TIME即为sector_TIME[f,s]
        sector_TIME[f,s]=DATA3[DATA3['sector']==s]['MIN'].values[0]
#print(sector_TIME)
#print(len(sector_TIME))

## 航班：起降机场

In [21]:
#约束4_1【航班的起降机场】
flightid_O_D={}
for f in DATA1['flightid']:
    flightid_O_D[f]=DATA1[DATA1['flightid']==f]['s_depap'].values.tolist()+DATA1[DATA1['flightid']==f]['s_arrap'].values.tolist()
#print(flightid_O_D)
#print(len(flightid_O_D))

## 航班：航路飞行时间

In [22]:
#约束6【航班与航路飞行时间sfly】
flightid_sfly={}
for f in DATA1['flightid']:
    flightid_sfly[f]=DATA1[DATA1['flightid']==f]['SFLY'].values[0]
#print(flightid_sfly)
#print(len(flightid_sfly))

## FF航班：降落机场

In [23]:
#约束4_2【FF航班的降落机场】
flightid_FF_D={}
for f in DATA1_2['flightid']:
    flightid_FF_D[f]=DATA1[DATA1['flightid']==f]['s_arrap'].values.tolist()
#print(flightid_FF_D)
#print(len(flightid_FF_D))

## WF航班的起飞机场

In [24]:
#约束4_2【WF航班的起飞机场】
flightid_WF_O={}
for f in DATA1_1['flightid']:
    flightid_WF_O[f]=DATA1[DATA1['flightid']==f]['s_depap'].values.tolist()
#print(flightid_WF_O)
#print(len(flightid_WF_O))

## WF航班：航班计划中的起飞时间

In [25]:
#约束4_3【WF航班的计划起飞时间】
flightid_WF_Time_departure={}
for f in DATA1_1['flightid']:
    flightid_WF_Time_departure[f]=DATA1[DATA1['flightid']==f]['Time'].values[0]
#print(flightid_WF_Time_departure)
#print(len(flightid_WF_Time_departure))

## 所有航班：航班的计划起飞时间

In [26]:
#目标函数【所有航班与所有航班分配到的起飞时间】
#陆侧延误的WF航班的起飞时间
flightid_Time_departure={}
for f in DATA1['flightid']:
    flightid_Time_departure[f]=DATA1[DATA1['flightid']==f]['Time'].values[0]
#print(flightid_Time_departure)
#print(len(flightid_Time_departure))

## 所有航班：航班的计划降落时间

In [27]:
#目标函数【所有航班与所有航班分配到的起飞时间】
#陆侧延误的WF航班的起飞时间
flightid_Time_arrival={}
for f in DATA1['flightid']:
    flightid_Time_arrival[f]=DATA1[DATA1['flightid']==f]['ATime'].values[0]
#print(flightid_Time_arrival)
#print(len(flightid_Time_arrival))

## 原航班计划中的所有起飞机场：起飞机场的起飞航班集合

In [28]:
#得到DATA1_1中航班起飞机场的集合flightid_WF_airport_O_dep
flightid_WF_airport_O_dep=DATA1_1['s_depap'].value_counts().index.tolist()
#将flightid_WF_airport_O_dep变为集合
flightid_WF_airport_O_dep=set(flightid_WF_airport_O_dep)
#再将flightid_WF_airport_O_dep变为列表
flightid_WF_airport_O_dep=list(flightid_WF_airport_O_dep)
#print(flightid_WF_airport_O_dep)
#print(len(flightid_WF_airport_O_dep))
#得到起飞机场为a的WF航班
flightid_WF_airport_dep_plane={}
for a in flightid_WF_airport_O_dep:
    flightid_WF_airport_dep_plane[a]=DATA1_1[DATA1_1['s_depap']==a]['flightid'].values.tolist()
#print(flightid_WF_airport_dep_plane)
#print(len(flightid_WF_airport_dep_plane))

## 联程航班对【1段：2段】

In [29]:
liancheng={}
#得到DATA1中liancheng列不为0和1的行
DATA1_liancheng=DATA1[(DATA1['liancheng']!=0)&(DATA1['liancheng']!=1)]
#print(DATA1_liancheng)
#print(len(DATA1_liancheng))
#liancheng字典中key为liancheng列的值，value为flightid列的值
for i in range(len(DATA1_liancheng)):
    liancheng[DATA1_liancheng.iloc[i]['liancheng']]=DATA1_liancheng.iloc[i]['flightid']
#print(liancheng)
#print(len(liancheng))
#设置航班中转的最小时间间隔为30分钟，即为6个时间片
T_zhouzhuan=6
T_ZHOUZHUAN=36

## 联程航班对【1段降落机场，2段起飞机场】

In [30]:
DATA1_liancheng=DATA1[(DATA1['liancheng']!=0)&(DATA1['liancheng']!=1)]
#约束7的liancheng航班对P的tuplelist搭建
liancheng_tuplelist=gp.tuplelist((i,j) for (i,j) in zip(DATA1_liancheng['liancheng'],DATA1_liancheng['flightid']))
#print(liancheng_tuplelist)
#print(len(liancheng_tuplelist))  #tuplelist可以进行索引，含有多个列表的元组
#约束7的liancheng航班对P的起落机场对的tuplelist搭建
#用liancheng_tuplelist中的每个liancheng航班对的第1个航班号的降落机场和第2个航班号的起飞机场替换原本liancheng_tuplelist中的liancheng航班对
OD_liancheng_tuplelist=liancheng_tuplelist.copy()
for (i,j) in OD_liancheng_tuplelist:
    OD_liancheng_tuplelist[OD_liancheng_tuplelist.index((i,j))]=(DATA1[DATA1['flightid']==i]['s_arrap'].values[0],DATA1[DATA1['flightid']==j]['s_depap'].values[0])
#将OD_liancheng_tuplelist转化为tuplelist格式，就是和liancheng_tuplelist一样的格式
OD_liancheng_tuplelist=gp.tuplelist(OD_liancheng_tuplelist)
#print(OD_liancheng_tuplelist)
#建立liancheng_tuplelist与OD_liancheng_tuplelist的字典
liancheng_pair={}
for i in range(len(liancheng_tuplelist)):
    liancheng_pair[liancheng_tuplelist[i]]=OD_liancheng_tuplelist[i]
#print(liancheng_pair)
#得到liancheng_pair的keys列表
liancheng_pair_keys=list(liancheng_pair.keys())
#print(liancheng_pair_keys[0])
#得到liancheng_pair的va;ues列表
liancheng_pair_values=list(liancheng_pair.values())
#print(len(liancheng_pair_values))
#print(liancheng_pair_values)
#用f1，f2对liancheng_pair_values进行循环索引
#for (f1,f2) in [liancheng_pair_values[0]]:  #此处不加中括号就是2索1
    #print(f1,f2)

## 考虑容量的起飞机场：路过机场的航班

In [31]:
#约束8：建立起飞机场为a的机场航班号集合的字典，键为起飞机场，值为起飞机场为a的机场航班号列表
dep_flightid_dict={}
DEP_airport1=DEP_airport.copy()
#给DEP_airport1的每一个元素加上-O
DEP_airport1=[i+'-O' for i in DEP_airport1]
for a in DEP_airport1:#只考虑日均航班量大于200的机场
    dep_flightid_dict[a]=DATA1[DATA1['s_depap']==a]['flightid'].values.tolist()
#print(dep_flightid_dict)
#print(len(dep_flightid_dict))

## 考虑容量的降落机场：路过机场的航班

In [32]:
#约束9：建立到达机场为a的机场航班号集合的字典，键为到达机场，值为到达机场为a的机场航班号列表
arr_flightid_dict={}
ARR_airport1=ARR_airport.copy()
#给ARR_airport1的每一个元素加上-D
ARR_airport1=[i+'-D' for i in ARR_airport1]
for a in ARR_airport1:#只考虑日均航班量大于200的机场
    arr_flightid_dict[a]=DATA1[DATA1['s_arrap']==a]['flightid'].values.tolist()
#print(arr_flightid_dict)
#print(len(arr_flightid_dict))

## 考虑容量的扇区：路过扇区的航班

In [33]:
#约束10：建立shanqu_list飞行路径中包含s的航班号集合的字典，键为s，值为shanqu_list飞行路径中包含s的航班号列表
sector_flightid_dict={}
for s in shanqu_list:
    for f in DATA1['flightid']:
        #如果shanqu_list飞行路径中包含s，则将flightid加入到s对应的sector_flightid_dict字典中
        if s in flightid_sector[f]:
            sector_flightid_dict[s]=sector_flightid_dict.get(s,[])+[f]
        else:
            continue
#print(sector_flightid_dict)
#print(len(sector_flightid_dict))

## FF航班在ST时刻所处的扇区；前序扇区

In [34]:
#约束1_1：计算航班、扇区：进扇区的时间的字典
#flightid_sector_jinshan={}
flightid_sector_ST={}
#约束1_3：计算航班、扇区：在ST之前的扇区的字典
flightid_sector_ST_qianxu={}
#对于DATA1中的每一个航班f，用s进行f对应flightid_sector的循环，以f、s为key值，DATA2的jinshan5min为value，建立字典
for f in DATA1_2['flightid']:
    for s in flightid_sector[f]:
        #如果f在s的进扇区时间在108之前且出扇时间在108之后或者进扇时间和出扇时间都是108
        if DATA2[(DATA2['flightid']==f)&(DATA2['sector']==s)]['jinshan5min'].values[0]<108:
            if DATA2[(DATA2['flightid']==f)&(DATA2['sector']==s)]['chushan5min'].values[0]>108:
                a=flightid_sector[f][flightid_sector[f].index(s)]
                flightid_sector_ST[f]=a
                flightid_sector_ST_qianxu[f]=flightid_sector[f][0:flightid_sector[f].index(s)]
                break
        else:
            if DATA2[(DATA2['flightid']==f)&(DATA2['sector']==s)]['jinshan5min'].values[0]==108:
                a=flightid_sector[f][flightid_sector[f].index(s)]
                flightid_sector_ST[f]=a
                flightid_sector_ST_qianxu[f]=flightid_sector[f][0:flightid_sector[f].index(s)]
                break
#print(flightid_sector_ST)
#print(len(flightid_sector_ST))
#print(flightid_sector_ST_qianxu)
#print(len(flightid_sector_ST_qianxu))

## 检验上一步是否有漏网之鱼

In [35]:
#确保航班计划里的FF航班都有对应的扇区路径【确保上一个单元格是对的】
#打印DATA1_2['flightid']的长度
AAAA=DATA1_2['flightid']
#化AAAA为列表
AAAA=AAAA.values.tolist()
#print(len(AAAA))
#print(AAAA)
#BBBB为flightid_sector_ST的keys
BBBB=flightid_sector_ST.keys()
#化BBBB为列表
BBBB=list(BBBB)
#print(len(BBBB))
#print(BBBB)
#CCCC为AAAA与BBBB的差集
CCCC=list(set(AAAA).difference(set(BBBB)))
#print(len(CCCC))
#print(CCCC)

## FF航班的前序扇区：FF航班进入扇区的时间

In [36]:
#计算FF航班进入前序扇区的时间
flightid_sector_shijian={}
#对flightid_sector_ST_qianxu的key值进行循环
for f in flightid_sector_ST_qianxu.keys():
    #对flightid_sector_ST_qianxu的value值进行循环
    for s in flightid_sector_ST_qianxu[f]:
        #flightid_sector_shijian的key值为f、s，value值为DATA2中的jinshan5min
        flightid_sector_shijian[f,s]=DATA2[(DATA2['flightid']==f)&(DATA2['sector']==s)]['jinshan5min'].values[0]
#print(flightid_sector_shijian)
#print(len(flightid_sector_shijian))

## 定义机会约束的转化函数

In [37]:
######不确定性约束中容量确定化的函数
#定义起飞机场不确定容量的机会约束转化函数，利用起飞机场容量分布律将其转化为确定容量的机会约束
#其中a为起飞机场，T为15min的时间片
def capicity_airport_dep_zhuanhua(a,T,MMM):
    #在capacity_dep_zuizhongjieguo中得到机场为a，时刻为T的部分
    A=capacity_dep_zuizhongjieguo[capacity_dep_zuizhongjieguo['jichang']==a][capacity_dep_zuizhongjieguo['shike']==T]
    #将累加概率与机会约束从最后一行开始比大小，得到首个累加概率大于机会约束的行对应的不确定容量值
    for i in range(len(A)-1,-1,-1):
        if A['leijiagailv'].values[i]>=A[MMM].values[i]:  #若要化为确定性约束，则需要将‘累加概率’化为‘确定容量概率’
            return A['buquedingxingrongliang'].values[i]                  #若要化为确定性约束，则需要将‘确定容量’化为‘确定容量’
            break
        
#定义降落机场不确定容量的机会约束转化函数，利用降落机场容量分布律将其转化为确定容量的机会约束
#其中a为降落机场，T为15min的时间片
def capicity_airport_arr_zhuanhua(a,T,MMM):
    #在capacity_arr_zuizhongjieguo中得到机场为a，时刻为T的部分
    A=capacity_arr_zuizhongjieguo[capacity_arr_zuizhongjieguo['jichang']==a][capacity_arr_zuizhongjieguo['shike']==T]
    #将累加概率与机会约束从最后一行开始比大小，得到首个累加概率大于机会约束的行对应的不确定容量值
    for i in range(len(A)-1,-1,-1):
        if A['leijiagailv'].values[i]>=A[MMM].values[i]:  #若要化为确定性约束，则需要将‘累加概率’化为‘确定容量概率’
            return A['buquedingxingrongliang'].values[i]                  #若要化为确定性约束，则需要将‘确定容量’化为‘确定容量’
            break

#定义扇区不确定容量的机会约束转化函数，利用扇区容量分布律将其转化为确定容量的机会约束
#其中s为扇区，T为15min的时间片
def capicity_sector_zhuanhua(s,T,MMM):
    #在capacity_shanqu_zuizhongjieguo中得到扇区为s，时刻为T的部分
    A=capacity_shanqu_zuizhongjieguo[capacity_shanqu_zuizhongjieguo['shanqu']==s][capacity_shanqu_zuizhongjieguo['shike']==T]
    #将累加概率与机会约束从最后一行开始比大小，得到首个累加概率大于机会约束的行对应的不确定容量值
    for i in range(len(A)-1,-1,-1):
        if A['leijiagailv'].values[i]>=A[MMM].values[i]:  #若要化为确定性约束，则需要将‘累加概率’化为‘确定容量概率’
            return A['buquedingxingrongliang'].values[i]                  #若要化为确定性约束，则需要将‘确定容量’化为‘确定容量’
            break

######确定容量约束中容量确定化的函数【以下def都会产生固定值】
#其中a为起飞机场，T为15min的时间片
def capicity_airport_dep_zhuanhua_queding(a,T):
    #在capacity_dep_zuizhongjieguo中得到机场为a，时刻为T的部分
    A=capacity_dep_zuizhongjieguo[capacity_dep_zuizhongjieguo['jichang']==a][capacity_dep_zuizhongjieguo['shike']==T]
    #将确定容量概率与机会约束从最后一行开始比大小，得到首个确定容量概率大于机会约束的行对应的确定容量值
    for i in range(len(A)-1,-1,-1):
        if A['quedingxingronglianggailv'].values[i]>=1:
            return A['quedingxingrongliang'].values[i]
            break

#其中a为降落机场，T为15min的时间片
def capicity_airport_arr_zhuanhua_queding(a,T):
    #在capacity_arr_zuizhongjieguo中得到机场为a，时刻为T的部分
    A=capacity_arr_zuizhongjieguo[capacity_arr_zuizhongjieguo['jichang']==a][capacity_arr_zuizhongjieguo['shike']==T]
    #将确定容量概率与机会约束从最后一行开始比大小，得到首个确定容量概率大于机会约束的行对应的确定容量值
    for i in range(len(A)-1,-1,-1):
        if A['quedingxingronglianggailv'].values[i]>=1:
            return A['quedingxingrongliang'].values[i]
            break
        

#其中s为扇区，T为15min的时间片
def capicity_sector_zhuanhua_queding(s,T):
    #在capacity_shanqu_zuizhongjieguo中得到扇区为s，时刻为T的部分
    A=capacity_shanqu_zuizhongjieguo[capacity_shanqu_zuizhongjieguo['shanqu']==s][capacity_shanqu_zuizhongjieguo['shike']==T]
    #将确定容量概率与机会约束从最后一行开始比大小，得到首个确定容量概率大于机会约束的行对应的确定容量值
    for i in range(len(A)-1,-1,-1):
        if A['quedingxingronglianggailv'].values[i]>=1:
            return A['quedingxingrongliang'].values[i]
            break



# 建立模型

In [38]:
class MAINMODEL:
    def __init__(self,iii,MMM,CG,CA):
        self.model=gp.Model()
        self.III=iii
        self.M=MMM
        self.C_g=CG
        self.C_a=CA
        
        
    def add_var_tuplelist(self):
        #其中f为DATA1中flightid列的遍历，s为f在DATA1中对应的OD在DATA2中对应的OD的sector列的遍历
        #【包括起降机场，所以时间片开始时间为[最早航班的起飞时间,最晚航班降落时间]】【W_fst与X_fat的时间片跨度是相同的，因为都包括起降机场】
        self.var_W_fst=gp.tuplelist([(f,s,t) 
                            for f in DATA1['flightid'] 
                            for s in flightid_sector[f] 
                            for t in range(52,247)])  #################原本是[0,247]，考虑飞机起降时间区间和最大延误时间，为简化不必要的模型规模将其缩小到[52,247]
        self.var_X_fat=gp.tuplelist([(f,a,t)
                            for f in DATA1['flightid'] 
                            for a in flightid_O_D[f] 
                            for t in range(52,247)])  #################原本是[0,247]，考虑飞机起降时间区间和最大延误时间，为简化不必要的模型规模将其缩小到[52,247]
        self.var_b_fsT=gp.tuplelist([(f,s,T)
                            for s in shanqu_list
                            for f in sector_flightid_dict[s] 
                            for T in range(36,61)])  #这里的时间片跨度为15min
        #添加的决策变量Airdelay_f是连续型变量【用于描述目标函数中的空侧延误】
        self.var_Airdelay_f=gp.tuplelist([(f)
                                        for f in DATA1['flightid']])
        #添加的决策变量P_Airdelay_f是指示约束变量【用于描述目标函数中的空侧延误】
        self.var_P_Airdelay_f=gp.tuplelist([(f)
                                        for f in DATA1['flightid']])
        
        
    def add_variables(self):
        self.W_fst=self.model.addVars(self.var_W_fst,lb=0,ub=1,vtype=gp.GRB.BINARY,name='W_fst')
        self.X_fat=self.model.addVars(self.var_X_fat,lb=0,ub=1,vtype=gp.GRB.BINARY,name='X_fat')
        self.b_fst=self.model.addVars(self.var_b_fsT,lb=0,ub=1,vtype=gp.GRB.BINARY,name='b_fst')
        #添加的决策变量Airdelay_f是连续型变量，且取值由负无穷到正无穷
        self.Airdelay_f=self.model.addVars(self.var_Airdelay_f,lb=-gp.GRB.INFINITY, ub=gp.GRB.INFINITY,vtype=gp.GRB.CONTINUOUS,name='Airdelay_f')
        self.P_Airdelay_f=self.model.addVars(self.var_P_Airdelay_f,lb=0,ub=1,vtype=gp.GRB.BINARY,name='P_Airdelay_f')
        
        
    def add_constraints(self):
        #（已审，1-3已经包含了1-1,1-1没有约束力）
        #约束1-1：在模型开始进行战术优化的ST【时间片108】时刻，每个已起飞飞机所处扇区的W_fst值限制为1
        self.model.addConstrs((self.W_fst[f,s,ST]==1 
                                for f in DATA1_2['flightid'] #FF航班
                                for s in [flightid_sector_ST[f]]),name='C1-1') #一个f对应一个s

        #（已审）约束1-2：将已起飞航班FF的起飞时刻的【X_fat】设置为1【这里只用设置一个1就可以，约束4可以对剩下t对应的X_fat进行约束】
        self.model.addConstrs((self.X_fat[f,s,t]==1 
                                for f in DATA1_2['flightid'] #FF航班
                                for s in DATA1[DATA1['flightid']==f]['s_depap'].values.tolist() #FF航班的起飞机场
                                for t in DATA1[DATA1['flightid']==f]['Time'].values.tolist()),name='C1-2') 

        #（已审）约束1-3：将已起飞航班FF的【起飞机场以及前序扇区】的【W_fst】值"进行设置【仿真时其实用不上，从9点开始即可】
        self.model.addConstrs((self.W_fst[f,s,t]==1
                                for f in DATA1_2['flightid']
                                for s in flightid_sector_ST_qianxu[f]
                                for t in range(flightid_sector_shijian[f,s],247)),name='C1-3')

        #（已审）约束2：描述所有航班的决策变量W_fst的时间关系【起降机场和扇区】
        self.model.addConstrs((self.W_fst[f,s,t-1]<=self.W_fst[f,s,t] 
                            for f in DATA1['flightid'] 
                            for s in flightid_sector[f]
                            for t in range(53,247)),name='C2')  #这里的时间我将其拓展到了所有可行的时间片

        #（已审）约束3：描述决策变量X_fat与W_fst的时间关系【起降机场】       
        self.model.addConstrs((self.X_fat[f,s,t]==self.W_fst[f,s,t]-self.W_fst[f,s,t-1] 
                            for f in DATA1['flightid'] 
                            for s in flightid_O_D[f] #包含f的起飞机场和降落机场
                            for t in range(53,247)),name='C3')  #这里的时间我将其拓展到了所有可行的时间片

        #（已审）约束4_1：描述X_fat所有航班起降的唯一性【起降机场】【（65，245）】
        self.model.addConstrs((gp.quicksum(self.X_fat[f,s,t] for t in range(52,247))==1  #这里的时间我将其拓展到所有时间片
                            for f in DATA1['flightid'] 
                            for s in flightid_O_D[f]),name='C4-1')

        #（4—2似乎意义不大，因为下文有约束5）
        #（已审）约束4_2：描述FF航班的降落时间【由数据结构可知，其必然晚于或等于108时间片】
        self.model.addConstrs((gp.quicksum(self.X_fat[f,s,t] for t in range(108,247))==1 
                            for f in DATA1_2['flightid'] 
                            for s in flightid_FF_D[f]),name='C4-2')

        #（已审）约束4_3：描述WF航班的起飞时间【由数据结构可知，其必然处于108时间片与原计划起飞时间片最大延误时间片之间】
        self.model.addConstrs((gp.quicksum(self.X_fat[f,s,t] for t in range(108,flightid_WF_Time_departure[f]+37))==1 
                            for f in DATA1_1['flightid'] 
                            for s in flightid_WF_O[f]),name='C4-3')
        
        #（已审）约束4_4：描述所有航班的降落时间
        self.model.addConstrs((gp.quicksum(self.X_fat[f,s,t] for t in range(flightid_Time_arrival[f]-12,247))==1 
                     for f in DATA1['flightid']
                     for s in [flightid_s_arrap[f]]),name='C4-4')

        #（已审）约束4_5：描述所有航班的“到场时刻”晚于“离场时刻”
        self.model.addConstrs((gp.quicksum(self.X_fat[f,s,t]*t 
                                        for s in [flightid_s_arrap[f]]
                                        for t in range(52,247))>=
                            gp.quicksum(self.X_fat[f,s,t]*t 
                                        for s in [flightid_s_depap[f]]
                                        for t in range(52,247)) 
                                for f in DATA1['flightid']),name='C4-5')

        #（已审）约束5：扇区的最短飞行时间【针对所有扇区】
        self.model.addConstrs((self.W_fst[f,sector_NEXT[f,s],t+sector_TIME[f,s]]<=self.W_fst[f,s,t]
                            for f in DATA1['flightid']
                            #s为f在DATA1中对应的OD在DATA2中对应的OD的sector列的遍历【但不包括列表的最后一个元素降落机场】
                            for s in flightid_sector_nolastone[f]
                            for t in range(108,247-sector_TIME[f,s])),name='C5')

        #（已审）约束6_1：航班空中最大延误量的限制为1小时
        self.model.addConstrs(((gp.quicksum(self.X_fat[f,s,t]*t 
                                        for s in [flightid_s_arrap[f]] 
                                        for t in range(52,247))-
                            gp.quicksum(self.X_fat[f,s,t]*t 
                                        for s in [flightid_s_depap[f]] 
                                        for t in range(52,247))-
                            flightid_sfly[f])<=12 
                            for f in DATA1['flightid']),name='C6_1')

        #这个约束里的a1，a2，t1，t2并非有必要去写角标
        #（已审）约束6_2：决策变量Airdelay_f与其他决策变量的关系【通过计算得到Airdelay_f，以便目标函数进行使用】
        self.model.addConstrs((gp.quicksum(t1*self.X_fat[f,a1,t1] for a1 in [flightid_s_arrap[f]] for t1 in range(52,247))-
                                        gp.quicksum(t2*self.X_fat[f,a2,t2] for a2 in [flightid_s_depap[f]] for t2 in range(52,247))-
                                        flightid_sfly[f]==self.Airdelay_f[f]
                            for f in DATA1['flightid']),name='C6_2')

        #（已审）约束6_3：使用指示函数约束进行Airdelay_f的选择【当Airdelay_f>=0时其为1，否则为0】
        self.model.addConstrs((self.Airdelay_f[f]>=0.5-self.M*(1-self.P_Airdelay_f[f]) for f in DATA1['flightid']),name='C6_3')

        #（已审）约束6_4：使用指示函数约束进行Airdelay_f的选择【当Airdelay_f<=0时其为1，否则为0】
        self.model.addConstrs((self.Airdelay_f[f]<=0.5+self.M*self.P_Airdelay_f[f] for f in DATA1['flightid']),name='C6_4')

        #（已审）约束7：联程航班约束，其中f2为联程航班的第二段航班，s2为联程航班的第二段航班的起飞机场，t为联程航班的第一段航班的降落时间片
        for AA in range(0,len(liancheng_pair)):#这个约束之外的for对应的就是任意
            self.model.addConstrs((self.W_fst[f2,s2,t+6]<=self.W_fst[f1,s1,t] 
                                for (f1,f2) in [liancheng_pair_keys[AA]] 
                                for (s1,s2) in [liancheng_pair_values[AA]]
                                for t in range(52,241)),name='C7') #这两者之间的距离一直都应该保持6个时间片以上

        #（已审）约束8: 机会约束模型中的起飞机场容量约束【A为a或s对应的机会约束值】
        for T in range(int(ST/3),int(ET/3)+13):
            self.model.addConstrs((gp.quicksum(self.X_fat[f,a,t] 
                                               for t in range(3*T,3*T+3) 
                                               for f in dep_flightid_dict[a])  #这里的航班只包括考虑容量限制的起飞机场
                                   #小于等于机会约束值对应的机场容量【使用容量的离散分布律将机会约束确定化】
                                   <=capicity_airport_dep_zhuanhua(a,T,self.III) 
                                           for a in DEP_airport1),name='C8') #a中包含了需要考虑容量限制的起飞机场

        #（已审）约束9: 机会约束模型中的降落机场容量约束【A为a或s对应的机会约束值】
        for T in range(int(ST/3),int(ET/3)+13):                                                       
            self.model.addConstrs((gp.quicksum(self.X_fat[f,a,t] 
                                               for t in range(3*T,3*T+3) 
                                               for f in arr_flightid_dict[a])  #这里的航班只包括考虑容量限制的到达机场
                                   #小于等于机会约束值对应的机场容量【使用容量的离散分布律将机会约束确定化】
                                   <=capicity_airport_arr_zhuanhua(a,T,self.III)
                                           for a in ARR_airport1),name='C9') #a中包含了需要考虑容量限制的到达机场
        
        #（已审）约束10的指示函数约束
        #约束10_1:扇区约束的指示函数约束【大M法得到变量b的值】
        for T in range(int(ST/3),int(ET/3)+13):
            self.model.addConstrs((gp.quicksum(self.W_fst[f,s,t]-self.W_fst[f,sector_next[f,s],t] 
                                               for t in range(3*T,3*T+3))>=0.5-self.M*(1-self.b_fst[f,s,T]) 
                                   for s in shanqu_list  #这里的扇区只包括考虑容量限制的扇区
                                   for f in sector_flightid_dict[s]),name='C10_1')
        
        #（已审）约束10_2:扇区约束的指示函数约束【大M法得到变量b的值】
        for T in range(int(ST/3),int(ET/3)+13):
            self.model.addConstrs((gp.quicksum(self.W_fst[f,s,t]-self.W_fst[f,sector_next[f,s],t] 
                                               for t in range(3*T,3*T+3))<=0.5+self.M*self.b_fst[f,s,T] 
                                   for s in shanqu_list  #这里的扇区只包括考虑容量限制的扇区
                                   for f in sector_flightid_dict[s]),name='C10_2')
            
        #（已审）约束10_3: 机会约束模型中的扇区容量约束
        for T in range(int(ST/3),int(ET/3)+13):
            self.model.addConstrs((gp.quicksum(self.b_fst[f,s,T] 
                                               for f in sector_flightid_dict[s]) 
                                   #小于等于机会约束值对应的扇区容量【使用容量的离散分布律将机会约束确定化】
                                   <=capicity_sector_zhuanhua(s,T,self.III) 
                                   for s in shanqu_list),name='C10_3')


    #（已审）
    def set_objective(self):
        self.model.setObjective((gp.quicksum(self.C_g*self.X_fat[f,a,t]*abs(t-flightid_Time_departure[f]) 
                                            for a in flightid_WF_airport_O_dep 
                                            for f in flightid_WF_airport_dep_plane[a] 
                                            #t时间片为闭区间108-180（144时间片最多可以延后3小时，所以有180）【针对WF未离港航班必然在（108-180）进行离场，约束4_3也对其进行了WF航班离场时刻约束】
                                            for t in range(ST,ET+37))+  #这里只记录晚点起飞的航班产生的地面成本【本来也应该这样，但这会鼓励WF航班前移,所以这里这里计算了WF航班起飞偏移的成本，前移后移都算】
                                gp.quicksum(self.C_a*self.Airdelay_f[f]*self.P_Airdelay_f[f]
                                            for f in DATA1['flightid'])),
                                gp.GRB.MINIMIZE)
        
        
    def set_parameter(self):
        self.model.Params.TimeLimit = 144000

    
    def optimize(self):
        self.model.optimize()
        # #当目标函数值小于103000时
        # if self.model.ObjVal<102000:
        # #舍弃当前计算分支
        #     self.model.terminate()#终止当前计算分支，进行下一个计算分支
    
    def get_solution(self):
        self.model.write('../output_sol/output{}-288.sol'.format(self.III))#10可以换为{}，进行循环
        #print(self.model.ObjVal)#打印目标函数值
        #self.model.write('../output_sol/output0-288.sol')#10可以换为{}，进行循环

    def get_csv(self):
        self.AAA=[]
        #把所有变量存放到list中
        for v in self.model.getVars():
            self.AAA.append(v)
            
        #W不进行输出，数据量过大，作图用不上【在output0-247.sol中可以看到W的取值】
        #self.df_W_fst = pd.DataFrame(columns=['f','s','t','W_fst'])
        #for i in range(0,len(self.AAA)):
        #    AAAAA=self.AAA[i]
        #    AAAA=AAAAA.varname
        #    AA=AAAA[0]
        #    if AA=='W' and AAAAA.x==1:
        #        self.df_W_fst.loc[len(self.df_W_fst)]=[AAAA[6:-1].split(',')[0],
        #                                            AAAA[6:-1].split(',')[1],
        #                                            AAAA[6:-1].split(',')[2],
        #                                            AAAAA.x]
        #self.df_W_fst.to_csv('../W/0W_fst{}-247.csv'.format(self.III),index=False)#10可以换为{}，进行循环

        self.df_X_fat = pd.DataFrame(columns=['f','a','t','X_fat'])
        for i in range(0,len(self.AAA)):
            BBBBB=self.AAA[i]
            BBBB=BBBBB.varname
            BB=BBBB[0]
            if BB=='X' and BBBBB.x==1:
                self.df_X_fat.loc[len(self.df_X_fat)]=[BBBB[6:-1].split(',')[0],
                                                    BBBB[6:-1].split(',')[1],
                                                    BBBB[6:-1].split(',')[2],
                                                    BBBBB.x]
        self.df_X_fat.to_csv('../X/0X_fat{}-288.csv'.format(self.III),index=False)#10可以换为{}，进行循环
        self.df_X_fat.to_csv('../X/0X_fat0-288.csv',index=False)#10可以换为{}，进行循环

        self.df_b_fst = pd.DataFrame(columns=['f','s','t','b_fst'])
        for i in range(0,len(self.AAA)):
            CCCCC=self.AAA[i]
            CCCC=CCCCC.varname
            CC=CCCC[0]
            if CC=='b' and CCCCC.x==1:
                self.df_b_fst.loc[len(self.df_b_fst)]=[CCCC[6:-1].split(',')[0],
                                                    CCCC[6:-1].split(',')[1],
                                                    CCCC[6:-1].split(',')[2],
                                                    CCCCC.x]
        self.df_b_fst.to_csv('../b/0b_fst{}-288.csv'.format(self.III),index=False)#10可以换为{}，进行循环
        self.df_b_fst.to_csv('../b/0b_fst0-288.csv',index=False)#10可以换为{}，进行循环

        self.df_Airdelay_f = pd.DataFrame(columns=['f','Airdelay_f'])
        for i in range(0,len(self.AAA)):
            DDDDD=self.AAA[i]
            DDDD=DDDDD.varname
            DD=DDDD[0]
            if DD=='A':
                self.df_Airdelay_f.loc[len(self.df_Airdelay_f)]=[DDDD[11:-1].split(',')[0],DDDDD.x]
        self.df_Airdelay_f.to_csv('../A/0Airdelay_f{}-288.csv'.format(self.III),index=False)#10可以换为{}，进行循环
        self.df_Airdelay_f.to_csv('../A/0Airdelay_f0-288.csv',index=False)#10可以换为{}，进行循环

        self.df_P_Airdelay_f = pd.DataFrame(columns=['f','P_Airdelay_f'])
        for i in range(0,len(self.AAA)):
            EEEEE=self.AAA[i]
            EEEE=EEEEE.varname
            EE=EEEE[0]
            if EE=='P':
                self.df_P_Airdelay_f.loc[len(self.df_P_Airdelay_f)]=[EEEE[13:-1].split(',')[0],EEEEE.x]
        self.df_P_Airdelay_f.to_csv('../P_A/0P_Airdelay_f{}-288.csv'.format(self.III),index=False)#10可以换为{}，进行循环
        self.df_P_Airdelay_f.to_csv('../P_A/0P_Airdelay_f0-288.csv',index=False)#10可以换为{}，进行循环


# 模型运行

In [39]:
for iii in range(6,11):
    ceshi=MAINMODEL(iii,10000,50,100)
    #进行变量tuplelist的生成
    ceshi.add_var_tuplelist()
    #进行model的决策变量生成
    ceshi.add_variables()
    ceshi.set_objective()
    ceshi.add_constraints()
    ceshi.set_parameter()
    ceshi.optimize()
    ceshi.get_solution()
    ceshi.get_csv()

GurobiError: License expired 2024-12-19